# Analyze Car Reviews with LLMs

# Project goal

This is a project by [Datacamp](https://app.datacamp.com/) and I use it as an exercise for analyzing and translating reviews with transformers.

**Car-ing is sharing**, an auto dealership company for car sales and rental, is taking their services to the next level thanks to **Large Language Models (LLMs)**. They need to prototype a chatbot app with multiple functionalities that not only assist customers but also provide support to human agents in the company.

The solution should receive textual prompts and use a variety of pre-trained Hugging Face LLMs to respond to a series of tasks, e.g. classifying the sentiment in a car’s text review, answering a customer question, summarizing or translating text, etc.

In [27]:
# Import statements
import pandas as pd
import torch

import evaluate # Load accuracy and F1 score metrics    

from transformers import pipeline # Load a sentiment analysis LLM into a pipeline

from transformers import AutoTokenizer

In [28]:
# Load data
df = pd.read_csv('data/car_reviews.csv', delimiter = ';')

# Inspect data
df.head()

,Review,Class
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE
1,The car is fine. It's a bit loud and not very ...,NEGATIVE
2,"My first foreign car. Love it, I would buy ano...",POSITIVE
3,I've come across numerous reviews praising the...,NEGATIVE
4,I've been dreaming of owning an SUV for quite ...,POSITIVE


In [29]:
# Create a list with the reviews
reviews = df['Review'].tolist()

# Create a list with the sentiment labels
real_labels = df['Class'].tolist()

# Sentiment classification of car reviews

I load a sentiment analysis LLM to classify the sentiment of each review in the dataset into POSITIVE or NEGATIVE, and utilize the real labels to calculate the accuracy and F1 score of predictions. I use the 'distilbert-base-uncased-finetuned-sst-2-english' model wich is a pre-trained LLM for binary sentiment classification.

In [30]:
# Use 'distilbert-base-uncased-finetuned-sst-2-english', pre-trained LLM for binary sentiment classification
classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

# Perform inference on the car reviews and display prediction results
predicted_labels = classifier(reviews)
for review, prediction, label in zip(reviews, predicted_labels, real_labels):
    print(f"Review: {review}\nActual Sentiment: {label}\nPredicted Sentiment: {prediction['label']} (Confidence: {prediction['score']:.4f})\n")

c:\Users\tkirov\AppData\Local\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Review: I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use. Camping, road trips, etc. We dont have any children so I store most of the seats in my warehouse. I wanted the passenger van for the rear air conditioning. We drove our van from Florida to California for a Cross Country trip in 2014. We averaged about 18 mpg. We drove thru a lot of rain and It was a very comfortable and stable vehicle. The V8 Nissan Titan engine is a 500k mile engine. It has been tested many times by delivery and trucking companies. This is why Nissan gives you a 5 year or 100k mile bumper to bumper warranty. Many people are scared about driving this van because of its size. But with front and rear sonar sensors, large mirrors and the back up camera. It is easy to drive. The front and rear sensors also monitor the front and rear sides of the bumpers making it easier to park close to objects. Our Nissan NV is a Tow Monster. It pulls our 5000 pound travel tr

In [31]:
# Map categorical sentiment labels into integer {0,1} labels
references = [1 if label == 'POSITIVE' else 0 for label in real_labels]
predictions = [1 if label['label'] == 'POSITIVE' else 0 for label in predicted_labels]

In [32]:
# Use accuracy and F1 score metrics to evaluate the classificiation
accuracy = evaluate.load('accuracy')
f1 = evaluate.load('f1')

# Calculate accuracy and F1 score
# The metrics in the evaluate library take two arguments: 
# references, containing the ground truth labels, 
# and predictions, containing the classification outputs produced by the model.
accuracy_result_dict = accuracy.compute(references=references, predictions=predictions)
accuracy_result = accuracy_result_dict['accuracy']
f1_result_dict = f1.compute(references=references, predictions=predictions)
f1_result = f1_result_dict['f1']
print(f'Accuracy: {accuracy_result}')
print(f'F1 result: {f1_result}')

Accuracy: 0.8
F1 result: 0.8571428571428571


# Translation

I load an English-to-Spanish translation LLM ('Helsinki-NLP/opus-mt-en-es') to translate part of the first car review in the dataset into Spanish, and use the reference translations provided in a separate file to calculate the BLEU score.

Translation LLMs sequentially generate the translated output token by token. As an alternative to manually extracting a priori the first two sentences from the original English review, I play with the max_length argument when calling the model for inference. I try considering a maximum output length ranging between 25 and 30 tokens.

In [33]:
# Load translation LLM into a pipeline and translate car review
first_review = reviews[0]
translator = pipeline('translation', model='Helsinki-NLP/opus-mt-en-es')
translated_review = translator(first_review, max_length=27)[0]['translation_text']
print(f'Model translation:\n{translated_review}')

c:\Users\tkirov\AppData\Local\anaconda3\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
c:\Users\tkirov\AppData\Local\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Your input_length: 365 is bigger than 0.9 * max_length: 27. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


Model translation:
Estoy muy satisfecho con mi 2014 Nissan NV SL. Uso esta furgoneta para mis entregas de negocios y uso personal.


In [34]:
# Load reference translations from file
with open('data/reference_translations.txt', 'r') as file:
    lines = file.readlines()
references = [line.strip() for line in lines]
print(f'Spanish translation references:\n{references}')

# Load and calculate BLEU score metric
bleu = evaluate.load('bleu')
bleu_score = bleu.compute(predictions=[translated_review], references=[references])
print(bleu_score['bleu'])

Spanish translation references:
['Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.', 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.']
0.6022774485691839


# Extractive QA

I load a pre-trained LLM for extractive QA ('deepset/minilm-uncased-squad2'), that takes two inputs (question and context) and extracts an answer directly from the context.

Since I opt for using auto classes for loading the model and tokenizer, I need to post-process the raw outputs by:
* Identifying the start and end token positions of the answer, given by maximum logits in outputs.start_logits and outputs.end_logits (assuming the raw model outputs have been stored in outputs).
* Extracting and decoding an answer span delimited by the previously identified start-end positions.

In [35]:
# Import auto classes (optional: can be solved via pipelines too)
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering

# Instantiate model and tokenizer
model_ckp = 'deepset/minilm-uncased-squad2'
tokenizer = AutoTokenizer.from_pretrained(model_ckp)
model = AutoModelForQuestionAnswering.from_pretrained(model_ckp)

# Define context and question, and tokenize them
context = reviews[1]
print(f'Context:\n{context}')
question = 'What did he like about the brand?'
inputs = tokenizer(question, context, return_tensors='pt')

# Perform inference and extract answer from raw outputs
with torch.no_grad():
  outputs = model(**inputs)
start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits) + 1
answer_span = inputs['input_ids'][0][start_idx:end_idx]

# Decode and show answer
answer = tokenizer.decode(answer_span)
print('Answer: ', answer)

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Context:
The car is fine. It's a bit loud and not very powerful. On one hand, compared to its peers, the interior is well-built. The transmission failed a few years ago, and the dealer replaced it under warranty with no issues. Now, about 60k miles later, the transmission is failing again. It sounds like a truck, and the issues are well-documented. The dealer tells me it is normal, refusing to do anything to resolve the issue. After owning the car for 4 years, there are many other vehicles I would purchase over this one. Initially, I really liked what the brand is about: ride quality, reliability, etc. But I will not purchase another one. Despite these concerns, I must say, the level of comfort in the car has always been satisfactory, but not worth the rest of issues found.
Answer:  ride quality, reliability


# Summarize and analyze a car review

I load a summarization LLM and pass it the last car review in the dataset to generate a summary of approximately 50-55 tokens. I analyze if there are any potential biases in the generated summary text by calculating the maximum toxicity and regard metrics.

I format metric inputs as a list. When I extract the text of a single output generated by an LLM and pass it as an argument to the compute() method of the evaluate library, some metrics often require encapsulating that single string into a 1-element list. For instance, use predictions=[summarized_text], with summarized_text as the string variable containing the extracted summarized text from the LLM output.

In [36]:
# Get original text to summarize upon car review
text_to_summarize = reviews[-1]
print(f'Original text:\n{text_to_summarize}')

# Load summarization pipeline and perform inference
model_name = 'cnicu/t5-small-booksum'
summarizer = pipeline('summarization', model=model_name)
outputs = summarizer(text_to_summarize, max_length=53)
summarized_text = outputs[0]['summary_text']
print(f'Summarized text:\n{summarized_text}')

Original text:
I've been dreaming of owning an SUV for quite a while, but I've been driving cars that were already paid for during an extended period. I ultimately made the decision to transition to a brand-new car, which, of course, involved taking on new payments. However, given that I don't drive extensively, I was inclined to avoid a substantial financial commitment. The Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more. I am VERY satisfied overall. I find myself needing to exercise extra caution when making lane changes, particularly owing to the blind spots resulting from the small side windows situated towards the rear of the vehicle. To address this concern, I am actively engaged in making adjustments to my mirrors and consciously reducing the frequency of la